In [14]:
from bs4 import BeautifulSoup
import requests


In [15]:
url = 'https://www.canada.ca/en/health-canada/corporate/transparency/corporate-management-reporting/evaluation.html'  #first page to scrape
response = requests.get(url)
content = response.content
soup = BeautifulSoup(content, 'html.parser') #load html of the main site

hrefs = []

In [16]:
eval_patterns = [
    'evaluation/results-',
    'evaluation/summary',
    'evaluation-reports/',
    'evaluations/',
    'reporting/summary',
    'reporting/evaluation',
    'publications.gc.ca/'
]
def eval_url(url, patterns=eval_patterns):
    for pattern in patterns:
        if pattern in url:
            return 'https://www.canada.ca/' + url
    return False
    

In [17]:
for link in soup.find_all('a'): #iterate through all links of the mainsite url (for each year)
    href = link.get('href') # get a list of sub-urls for reports for each year
    if href and (href := eval_url(href)):
        hrefs.append(href)

hrefs = list(set(hrefs)) #remove duplicates

In [32]:
from bs4 import BeautifulSoup
import requests

def load_html_text(url):
    # Make a request to the URL and retrieve the HTML content
    response = requests.get(url)
    content = response.content

    # Parse the HTML content
    soup = BeautifulSoup(content, 'html.parser')

    # Extract all the text
    section_identifier = 'main[property="mainContentOfPage"][resource="#wb-main"][typeof="WebPageElement"][class="container"]'
    section = soup.select_one(section_identifier) #select texts only from a specific section
    all_text = section.get_text()
    return all_text

with open("hc_evaluation_reports.txt", "w") as text_file:
  base_dir = '../data/hc/'
  success = 0
  for i, url in enumerate(hrefs):
    try:
      text_content = load_html_text(url)
    except Exception as e:
      print(f'{str(i).zfill(3)} - Failed to load {url}')
      print(e)
      continue
    try:
      text_file.write(text_content) #write all evaluation reports into a single .txt file
      print(f'{str(i).zfill(3)} - Wrote {url}')
      try:
        fp = open(base_dir + str(i).zfill(3) + '.txt', 'w') #write each evaluation report into a separate .txt file
        fp.write(text_content)
        fp.close()
      except Exception as e:
        print(e)
      success += 1
    except Exception as e:
      print(f'{str(i).zfill(3)} - Failed to write {url}')
      print(e)
print(f'Successfully wrote {success}/{len(hrefs)} files')

000 - Wrote https://www.canada.ca//en/health-canada/corporate/transparency/corporate-management-reporting/evaluation/summary-health-care-policy-contribution-program-2013-2014-2017-2018.html
001 - Wrote https://www.canada.ca//en/health-canada/corporate/transparency/corporate-management-reporting/evaluation/results-covid-alert-national-covid-19-exposure-notification-app.html
002 - Wrote https://www.canada.ca//en/health-canada/corporate/transparency/corporate-management-reporting/evaluation/results-glance-horizontal-evaluation-legalization-strict-regulation-cannabis.html
003 - Wrote https://www.canada.ca//en/health-canada/corporate/transparency/corporate-management-reporting/evaluation/2011-2012-2014-2015-canadian-centre-substance-abuse-named-grant-activities.html
004 - Wrote https://www.canada.ca//en/health-canada/corporate/transparency/corporate-management-reporting/evaluation/at-a-glance-departmental-evaluation-plan-2022-2023-2026-2027.html
005 - Wrote https://www.canada.ca//en/health-